In [ ]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

ssc = StreamingContext(sc, 5)

lines = KafkaUtils.createDirectStream(ssc,
                                      topics=['Demo-Topic'],
                                      kafkaParams={ 'metadata.broker.list': 'server-pd:9092' })

lines.pprint()
# 데이터 처리

ssc.start()
ssc.awaitTermination()

In [2]:
ssc.stop()

In [ ]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

ssc = StreamingContext(sc, 5)

lines = KafkaUtils.createDirectStream(ssc,
                                      topics=['Demo-Topic'],
                                      kafkaParams={ 'metadata.broker.list': 'server-pd:9092' })

# 데이터 처리
# (None, ...)
rows = lines.map(lambda line: line[1].split(','))
rows.pprint()

ssc.start()
ssc.awaitTermination()

In [2]:
ssc.stop()

In [ ]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

def processRDD(rdd):
    row = rdd.map(lambda line: line[1].split(','))
    if row.count() > 0:
        df = spark.createDataFrame(row)
        df.show()
    else:
        print("no data received")

ssc = StreamingContext(sc, 5)

lines = KafkaUtils.createDirectStream(ssc,
                                      topics=['Demo-Topic'],
                                      kafkaParams={ 'metadata.broker.list': 'server-pd:9092' })

# 데이터 처리
# (None, ...)
lines.foreachRDD(processRDD)

ssc.start()
ssc.awaitTermination()

In [ ]:
ssc.stop()

In [ ]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

def processRDD(rdd):
    row = rdd.map(lambda line: line[1].split(','))
    if row.count() > 0:
        df = spark.createDataFrame(row, ["date", 'car_number', 'speed_pedal', 'break_pedal', 'steer_angle', 'direct_light', 'speed', 'area_number'])
        df.show()
    else:
        print("no data received")


ssc = StreamingContext(sc, 5)

lines = KafkaUtils.createDirectStream(ssc,
                                      topics=['SmartCar-Topic'],
                                      kafkaParams={ 'metadata.broker.list': 'server-pd:9092' })

# 데이터 처리
# (None, ...)
# lines.pprint()
lines.foreachRDD(processRDD)

ssc.start()
ssc.awaitTermination()

In [2]:
ssc.stop()

In [ ]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

# hbase에  'DriverCarInfo' 테이블을 만들고 다음을 실행 ( create 'DriverCarInfo', 'cf1' )

catalog = ''.join("""{
    "table":{ "namespace": "default", "name": "DriverCarInfo" },
    "rowkey": "key",
    "columns": {
        "key": { "cf": "rowkey", "col": "key", "type": "string" },
        "date": { "cf": "cf1", "col": "date", "type": "string" }, 
        "car_number": { "cf": "cf1", "col": "car_number", "type": "string" }, 
        "speed_pedal": { "cf": "cf1", "col": "speed_pedal", "type": "string" }, 
        "break_pedal": { "cf": "cf1", "col": "break_pedal", "type": "string" }, 
        "steer_angle": { "cf": "cf1", "col": "steer_angle", "type": "string" }, 
        "direct_light": { "cf": "cf1", "col": "direct_light", "type": "string" }, 
        "speed": { "cf": "cf1", "col": "speed", "type": "string" }, 
        "area_number": { "cf": "cf1", "col": "area_number", "type": "string" }
    }
}""".split())

def makeRow(line):
    row = line[1].split(',') # 'a,b,c,d' -> ['a', 'b', 'c', 'd']
    row2 = [ row[0] + "-" + row[1] ] + row # [ 'a' + '-' + 'b' ] + ['a', 'b', 'c', 'd'] -> ['a-b', 'a', 'b', 'c', 'd']
    return row2


def processRDD(rdd):
    row = rdd.map(makeRow) # row = rdd.map(lambda line: line[1].split(','))
    if row.count() > 0:
        df = spark.createDataFrame(row, ["key", "date", 'car_number', 'speed_pedal', 'break_pedal', 'steer_angle', 'direct_light', 'speed', 'area_number'])
        df.write \
          .option('catalog', catalog) \
          .format('org.apache.spark.sql.execution.datasources.hbase') \
          .save()
        
        print('data processed !!!!')
    else:
        print("no data received")


ssc = StreamingContext(sc, 5)

lines = KafkaUtils.createDirectStream(ssc,
                                      topics=['SmartCar-Topic'],
                                      kafkaParams={ 'metadata.broker.list': 'server-pd:9092' })

# 데이터 처리
# (None, ...)
# lines.pprint()
lines.foreachRDD(processRDD)

ssc.start()
ssc.awaitTermination()

no data received
no data received
no data received
no data received
no data received
no data received
no data received
no data received
no data received


In [2]:
ssc.stop()